<a href="https://colab.research.google.com/github/pnjha/NLP-Assignments/blob/master/nlp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive


In [0]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import copy
import math
from keras.optimizers import SGD
from keras.initializers import RandomUniform
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed, Bidirectional, Embedding
from keras.layers import RepeatVector
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, CuDNNLSTM, Input, Embedding, TimeDistributed, Flatten, Dropout, Activation
from keras.callbacks import ModelCheckpoint
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

In [5]:
X = pd.read_table('./NLA S20 - Assignment 2 Data/enghin/train.en', names=['src'])
Y = pd.read_table('./NLA S20 - Assignment 2 Data/enghin/train.hi', names=['dest'])
lines = pd.concat([X[:3000],Y[:3000]], axis=1)
print(len(lines))
lines = shuffle(lines)
lines.sample(10)

3000


,src,dest
972,Reiki attunement is done in every course .,रेकी शक्तिपात हर कोर्स मे किया जाता है ।
265,Do not let rain water collect near the ditches...,जहाँ तक संभव हो बरसाती पानी को घर के आस-पास गड...
1419,Do mountain biking for 45 minutes .,45 मिनट माउंटेन बाइकिंग करें ।
53,"Cancer erupts by our wrong practices , thought...","कैंसर हमारे गलत अचार , विचार व्यवहार और आहार स..."
483,Combined DTP vaccines have been used the world...,1940 से ही दुनिया भर में मिले - जुले ( कम्बाइन...
955,Reiki destroys the cause of disease from roots .,रेकी बीमारी के कारण को जड़ मूल से नष्ट करती हैं
741,Thus lacs of women and families can be saved f...,इस प्रकार लाखों महिलाओं एवं परिवारों को मानसिक...
2649,"By August 21 , 2004 a total of 34 cases had co...",21 अगस्त 2004 तक देश में पोलियो के कुल 34 मामल...
2634,The reason of polio disease is polio mellitus ...,पोलियो बीमारी का कारण पोलियोमलाइटिस नामक वायरस...
1162,Scientists have discovered a new species of ba...,वैज्ञानिकों ने एक ऐसी नई प्रजाति के बैक्टीरिया...


In [0]:
def process_data(data,append_char):
  data = data.apply(lambda x: x.lower())
  data = data.apply(lambda x: x.strip())
  data = data.apply(lambda x: re.sub("'", '', x))
  exclude = set(string.punctuation) # Set of all special characters
  data = data.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
  remove_digits = str.maketrans('', '', digits)
  data = data.apply(lambda x: x.translate(remove_digits))
  if append_char:
    data = data.apply(lambda x : '<SOS> '+ x + ' <EOS>')
  return data

In [85]:
lines.src = process_data(lines.src,False)
lines.dest = process_data(lines.dest,True)
# lines.dest.sample(10)

1447    <SOS> अगर पढ़ते वक्त वह अपनी किताब को आँखों के...
2708    <SOS> किसी भी व्यक्ति में सिस्टोलिक ब्लड प्रेश...
965     <SOS> अच्छा रेकी चैनल बनने के लिये इच्छा सभी क...
1105    <SOS> आइस पैक शरीर के तापमान को स्वाभाविक रूप ...
2220    <SOS> आम तौर पर इस प्रक्रिया में मरीज को बेहोश...
1119    <SOS> दिन में ८ से १० गिलास पानी पिएं । ज्यूस ...
1660    <SOS> एक किस्म की शराब और बीयर बनाने में भी इस...
937     <SOS> कम से कम खाइए  फिर भी अपचन न हो  इसके लि...
2211    <SOS> खासकर ऐसी वस्तु  जो बहुत सख्त ही और शरीर...
1160    <SOS> अमेरिकी वैज्ञानिकों ने अब ऐसा अस्थिमज्जा...
Name: dest, dtype: object

In [0]:
def get_vocab_list(data):
  vocab = []
  for line in data:
    temp = line.split()
    for item in temp:
      if item not in vocab:
        vocab.append(item)
  print(vocab)
  return vocab

In [98]:
src_vocab = get_vocab_list(lines.src)
dest_vocab = get_vocab_list(lines.dest)
print(src_vocab[:10])
print(dest_vocab[:10])
src_vocab = sorted(src_vocab)
# dest_vocab = sorted(dest_vocab)
print(src_vocab[:10])
print(dest_vocab[:10])

['the', 'formula', 'to', 'stay', 'constantly', 'healthy', 'in', 'today', 's', 'mechanical', 'era', 'this', 'is', 'only', 'natural', 'that', 'we', 'compare', 'various', 'organs', 'of', 'our', 'body', 'with', 'instruments', 'used', 'day', 'life', 'these', 'chemical', 'substances', 'are', 'called', 'medicines', 'besides', 'on', 'being', 'hereditarily', 'sensitive', 'attack', 'asthma', 'can', 'occur', 'coming', 'contact', 'increasing', 'polluted', 'elements', 'present', 'environment', 'allergic', 'smoke', 'or', 'other', 'chemicals', 'motor', 'vehicles', 'and', 'factories', 'many', 'sexually', 'transmitted', 'diseases', 'show', 'some', 'symptoms', 'initially', 'end', 'without', 'any', 'treatment', 'when', 'rays', 'light', 'gets', 'collected', 'before', 'retina', 'at', 'different', 'stages', 'age', 'women', 'have', 'combat', 'health', 'problems', 'usually', 'following', 'found', 'patients', 'diabetes', 'child', 'soon', 'bored', 'useless', 'work', 'by', 'which', 'danger', 'cancer', 'left', 'b

In [99]:
print(src_vocab[:10])
print(dest_vocab[:10]) 

['a', 'aagantu', 'abdomen', 'ability', 'able', 'abnormal', 'abnormality', 'abode', 'abortion', 'abortions']
['<SOS>', 'निरंतर', 'स्वस्थ्य', 'रहने', 'के', 'मूलमंत्र', '<EOS>', 'आज', 'मशीनी', 'युग']


In [0]:
# Max Length of source sequence
def get_max_sentence_size(data):
  max_len = 0
  for line in data:
    temp = line.split()
    max_len = max(max_len, len(temp))
  return max_len

In [101]:
src_max_len = get_max_sentence_size(lines.src)
dest_max_len = get_max_sentence_size(lines.dest)
src_max_len,dest_max_len

(84, 86)

In [102]:
src_vocab_len = len(src_vocab)
dest_vocab_len = len(dest_vocab) + 1
src_vocab_len, dest_vocab_len

(4711, 6289)

In [0]:
def get_word_to_index_dict(vocab):
  word_to_index = {}
  for i, word in enumerate(vocab):
    word_to_index[word] = i+1
  return word_to_index

In [0]:
src_word_to_index = get_word_to_index_dict(src_vocab)
dest_word_to_index = get_word_to_index_dict(dest_vocab)

In [105]:
print(src_word_to_index)
print(dest_word_to_index)

{'a': 1, 'aagantu': 2, 'abdomen': 3, 'ability': 4, 'able': 5, 'abnormal': 6, 'abnormality': 7, 'abode': 8, 'abortion': 9, 'abortions': 10, 'about': 11, 'above': 12, 'abrasion': 13, 'abscess': 14, 'absolutely': 15, 'absorbed': 16, 'absorber': 17, 'absorption': 18, 'abstain': 19, 'abstinence': 20, 'abundance': 21, 'abundant': 22, 'acacia': 23, 'accept': 24, 'acceptability': 25, 'accepted': 26, 'accessory': 27, 'accident': 28, 'accidents': 29, 'according': 30, 'accumulate': 31, 'accumulated': 32, 'accumulates': 33, 'accumulation': 34, 'acetylcholine': 35, 'ache': 36, 'achieve': 37, 'achieved': 38, 'achievement': 39, 'achievements': 40, 'acid': 41, 'acidity': 42, 'acids': 43, 'acquaintances': 44, 'acquire': 45, 'acrobats': 46, 'act': 47, 'action': 48, 'actions': 49, 'activate': 50, 'active': 51, 'activities': 52, 'activity': 53, 'actual': 54, 'actually': 55, 'acupressure': 56, 'acupuncture': 57, 'acute': 58, 'ad': 59, 'add': 60, 'added': 61, 'addiction': 62, 'adding': 63, 'addition': 64, '

In [0]:
def get_index_to_word_dict(vocab):
  index_to_word = {}
  for i, word in enumerate(vocab):
    index_to_word[i+1] = word
  return index_to_word  

In [0]:
src_index_to_word = get_index_to_word_dict(src_vocab)
dest_index_to_word = get_index_to_word_dict(dest_vocab)

In [108]:
print(src_index_to_word)
print(dest_index_to_word)

{1: 'a', 2: 'aagantu', 3: 'abdomen', 4: 'ability', 5: 'able', 6: 'abnormal', 7: 'abnormality', 8: 'abode', 9: 'abortion', 10: 'abortions', 11: 'about', 12: 'above', 13: 'abrasion', 14: 'abscess', 15: 'absolutely', 16: 'absorbed', 17: 'absorber', 18: 'absorption', 19: 'abstain', 20: 'abstinence', 21: 'abundance', 22: 'abundant', 23: 'acacia', 24: 'accept', 25: 'acceptability', 26: 'accepted', 27: 'accessory', 28: 'accident', 29: 'accidents', 30: 'according', 31: 'accumulate', 32: 'accumulated', 33: 'accumulates', 34: 'accumulation', 35: 'acetylcholine', 36: 'ache', 37: 'achieve', 38: 'achieved', 39: 'achievement', 40: 'achievements', 41: 'acid', 42: 'acidity', 43: 'acids', 44: 'acquaintances', 45: 'acquire', 46: 'acrobats', 47: 'act', 48: 'action', 49: 'actions', 50: 'activate', 51: 'active', 52: 'activities', 53: 'activity', 54: 'actual', 55: 'actually', 56: 'acupressure', 57: 'acupuncture', 58: 'acute', 59: 'ad', 60: 'add', 61: 'added', 62: 'addiction', 63: 'adding', 64: 'addition', 6

In [0]:
def reverse_train_sentences(data):
  l = data.values.tolist()
  t = []
  for item in l:
    p = item.strip().split()
    p.reverse()
    k = []
    for i in p:
      i = i.strip()
      if len(i) > 0:
        k.append(i)
    p = k
    p = " ".join(p)
    p = p.strip()
    t.append(p)
  df = pd.DataFrame({'data':t})
  return df.data  

In [110]:
# Train - Test Split
X, y = lines.src, lines.dest
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train_normal = copy.deepcopy(X_train)
X_train = reverse_train_sentences(X_train)
print(X_train.shape, X_test.shape)
X_train.sample(10)

(2700,) (300,)


2697                        hands dirty using from escape
2340    month every directorate the to report this pos...
1229    water this of lots drink it sieving after and ...
219     so do women pregnant rural of third one than m...
2117     it liking am doing am there am i saying start we
232     maximum the to increased has childbirths insti...
913     days for alive stays virus this b hepatitis wi...
1678    forms hole a then and occurs inflammation then...
439     fatal and dangerous prove can wish own s one b...
1402    system nervous peripheral of medium the throug...
Name: data, dtype: object

In [0]:
def generate_batch(X = X_train, y = y_train):
  batch_size = len(X)
  encoder_input_data = np.zeros((batch_size, src_max_len),dtype='float32')
  decoder_target_data = np.zeros((batch_size, dest_max_len, dest_vocab_len),dtype='float32')
  decoder_data = np.zeros((batch_size, dest_max_len),dtype='float32')
  
  for i, (input_text, target_text) in enumerate(zip(X,y)):
      for t, word in enumerate(input_text.split()):
          encoder_input_data[i, t] = src_word_to_index[word]
      for t, word in enumerate(target_text.split()):
        if t>0:
          decoder_target_data[i, t-1, dest_word_to_index[word]] = 1
        decoder_data[i,t] = dest_word_to_index[word]
  return [encoder_input_data, decoder_data], decoder_target_data

In [0]:
latent_dim = 400

In [113]:
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(input_dim = src_vocab_len,  output_dim = 400, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(input_dim = dest_vocab_len, output_dim = 400, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,initial_state=encoder_states)
decoder_dense = Dense(dest_vocab_len, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, None, 400)    1884400     input_15[0][0]                   
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, None, 400)    2515600     input_16[0][0]                   
___________________________________________________________________________________________

In [0]:
X, y = generate_batch(X_train, y_train)
xtest, ytest = generate_batch(X_test, y_test)

In [0]:
train_samples = len(X_train)
val_samples = len(X_test)
epochs = 30
batch_size = 64

In [116]:
model.fit(X,y,
          epochs=epochs,
          batch_size=batch_size,
          validation_data =(xtest,ytest))

Train on 2700 samples, validate on 300 samples
Epoch 1/30
2700/2700 [==============================] - 19s 7ms/step - loss: 6.2118 - acc: 0.1062 - val_loss: 5.9089 - val_acc: 0.1207
Epoch 2/30
2700/2700 [==============================] - 17s 6ms/step - loss: 5.5780 - acc: 0.1383 - val_loss: 5.7943 - val_acc: 0.1523
Epoch 3/30
2700/2700 [==============================] - 16s 6ms/step - loss: 5.3302 - acc: 0.1592 - val_loss: 5.6841 - val_acc: 0.1637
Epoch 4/30
2700/2700 [==============================] - 17s 6ms/step - loss: 5.1375 - acc: 0.1682 - val_loss: 5.6504 - val_acc: 0.1699
Epoch 5/30
2700/2700 [==============================] - 17s 6ms/step - loss: 4.9647 - acc: 0.1794 - val_loss: 5.6173 - val_acc: 0.1761
Epoch 6/30
2700/2700 [==============================] - 16s 6ms/step - loss: 4.8026 - acc: 0.1909 - val_loss: 5.5453 - val_acc: 0.1893
Epoch 7/30
2700/2700 [==============================] - 17s 6ms/step - loss: 4.6429 - acc: 0.2026 - val_loss: 5.5101 - val_acc: 0.1912
Epoch 8/

In [0]:
model.save_weights('nmt_weights.h5')

In [0]:
model.load_weights('nmt_weights.h5')

In [0]:
def calculate_bleu_score(actual_string, predicted_string):
  actual_string = copy.deepcopy(actual_string)
  predicted_string = copy.deepcopy(predicted_string)
  reference = re.split("\s",actual_string.strip())
  candidate = re.split("\s",predicted_string.strip())
  smoothie = SmoothingFunction().method4
  score = sentence_bleu(reference, candidate)#, smoothing_function=smoothie)
  return score

In [0]:
def decode_input_sentence(arr,inverse_mapping,skip=False):
  line = ""
  for i in range(len(arr)):
    if arr[i] == 0.0:
      break
    if  inverse_mapping[arr[i]] == "<SOS>" or inverse_mapping[arr[i]] == "<EOS>":
      continue
    line += inverse_mapping[arr[i]]+" "
    
  return line

In [0]:
def decode_model_output(model_output):

    states_value = encoder_model.predict(model_output)
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = dest_word_to_index['<SOS>']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = dest_index_to_word[sampled_token_index]

        if (sampled_char == '<EOS>' or len(decoded_sentence) > 50):
            break

        decoded_sentence += ' '+sampled_char
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        states_value = [h, c]

    return decoded_sentence

In [124]:
bleu_score = 0.0
print(len(xtest))
for i in range(len(xtest[0])):
  src_input_sentence = decode_input_sentence(xtest[0][i],src_index_to_word)
  # print(ytestp[i,:])
  dest_input_sentence = decode_input_sentence(xtest[1][i],dest_index_to_word,True)
  output_sentence = decode_model_output(xtest[0][i])
  print("Actual Source: ",src_input_sentence)
  print("Actual Target: ",dest_input_sentence)
  print("predicted Target: ",output_sentence)
  curr_bs = calculate_bleu_score(dest_input_sentence,output_sentence)
  bleu_score += curr_bs
  print("Current BLEU Score: ",curr_bs)
  # break
print("Overall BLEU Score: ", bleu_score/len(xtest[0]))

2
Actual Source:  type diabetes grows gradually 
Actual Target:  टाइप डायबिटीज धीरेधीरे विकसित होती है । 
predicted Target:   शरीर के द्वारा ही दिखाई देते हैं ।
Current BLEU Score:  0.5946035575013605
Actual Source:  several symptoms shows up due to cancer that gives the impressions of cancer 
Actual Target:  कैंसर के कारण ऐसे अनेक लक्षण दिखलाई पड़ने लगते हैं जो कैंसर होने का आभास देते हैं । 
predicted Target:   शरीर के द्वारा ही दिखाई देते हैं ।
Current BLEU Score:  0.5946035575013605
Actual Source:  in the lithoptysis technique the stones present in any part of the urinary tract are targeted with aim and then the stones are broken by the shower of high capacity waves 
Actual Target:  लीथोट्रिप्स विधि में मूत्रमार्ग के किसी भी हिस्से में मौजूद पथरी की अचूक निशानदेही की जाती है और फिर उस पर अति क्षमता वाली तरंगों की बौछार पथरी को तोड़ दिया जाता है । 
predicted Target:   में दर्द ।
Current BLEU Score:  0.7598356856515925
Actual Source:  now through this small incision a foldable lens is 

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Actual Source:  why the action site of these medicines are synapse and not the nerve cell itself to understand it will not be now difficult 
Actual Target:  इन दवाओं का कार्य स्थल स्वयं तंत्रिकाकोशिका न होकर सिनेप्स क्यों यह समझना अब कठिन नहीं होगा । 
predicted Target:   क्यों होता है ।
Current BLEU Score:  0.7071067811865476
Actual Source:  if still a prey to diabetes then the reason is this that they do nt have the right information about their diet 
Actual Target:  फिर भी यदि मधुमेह के शिकार हैं तो कारण यही है कि उन्हें अपने आहार के बारे में सही ज्ञान नहीं है । 
predicted Target:   डायबिटीज से हो सकता है ।
Current BLEU Score:  0.6389431042462724
Actual Source:  the harmony between food and habits is a must then only the effort towards the direction of reducing weight obesity becomes meaningful 
Actual Target:  खानपान के तौरतरीकों तथा व्यायाम शैली के बीच संतुलन होना जरूरी है तभी वजनमोटापा कम करने की दिशा में प्रयास सार्थक हो सकते हैं । 
predicted Target:   हार्मोनल परिवर्तन ।
Current